In [38]:
import os
import time
import torch
import torch as tc
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import numpy as np
import sys,os

In [2]:
import models
from models import weights_init

# Main code

In [3]:
#Libera as funcionalidades da biblioteca cudnn
cudnn.benchmark = True

use_gpu = torch.cuda.is_available()
#use_gpu = False
if use_gpu:
    print("You are using CUDA. If it is not what you want, manually set this as False!")

You are using CUDA. If it is not what you want, manually set this as False!


In [4]:
nc = 3
ngpu = 1
nz = 100
ngf = 64
ndf = 64
n_extra_d = 0
n_extra_g = 1 # Aqui a jogada é que o gerador deve ser mais poderoso q o detetive
imageSize = 64

In [5]:
!cd images && pwd


/home/gabriel/Redes Neurais/Projeto_Final_GANS/Tutorial_2/images


## Setando as transformações

In [6]:
!ls images/images2/

ls: cannot access 'images/images2/': No such file or directory


In [7]:
!ls dataset

anime-faces  min_anime-faces  teste


In [41]:
dataset = dset.ImageFolder(
    root='/home/gabriel/Redes Neurais/Projeto_Final_GANS/Tutorial_2/dataset/min_anime-faces',
    transform=transforms.Compose([
            transforms.Scale((imageSize, imageSize)),
            # transforms.CenterCrop(opt.imageSize),
            transforms.ToTensor(),
            #transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)), # bring images to (-1,1)
        ])
)

## Setando o Dataloader

In [42]:
batch_size=64
num_workers = 4

In [43]:

dataloader = tc.utils.data.DataLoader(dataset, batch_size=64, shuffle=True, num_workers=1)
#126 classes

## Setando o modelo

### Modelo DCGAN

In [16]:
class _netD_1(nn.Module):
    def __init__(self, ngpu, nz, nc, ndf,  n_extra_layers_d):
        super(_netD_1, self).__init__()
        self.ngpu = ngpu
        main = nn.Sequential(
            # input is (nc) x 96 x 96
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False), # 5,3,1 for 96x96
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
        )

        # Extra layers
        for t in range(n_extra_layers_d):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, ndf * 8),
                            nn.Conv2d(ndf * 8, ndf * 8, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, ndf * 8),
                            nn.BatchNorm2d(ndf * 8))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, ndf * 8),
                            nn.LeakyReLU(0.2, inplace=True))


        main.add_module('final_layers.conv', nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False))
        main.add_module('final_layers.sigmoid', nn.Sigmoid())
        # state size. 1 x 1 x 1
        self.main = main

    def forward(self, input):
        gpu_ids = None
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            gpu_ids = range(self.ngpu)
        output = nn.parallel.data_parallel(self.main, input, gpu_ids)
        return output.view(-1, 1)


In [17]:
# DCGAN model, fully convolutional architecture
class _netG_1(nn.Module):
    def __init__(self, ngpu, nz, nc , ngf, n_extra_layers_g):
        super(_netG_1, self).__init__()
        self.ngpu = ngpu
        #self.nz = nz
        #self.nc = nc
        #self.ngf = ngf
        main = nn.Sequential(
            # input is Z, going into a convolution
            # state size. nz x 1 x 1
            nn.ConvTranspose2d(     nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ngf) x 32 x 32
        )

        # Extra layers
        for t in range(n_extra_layers_g):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, ngf),
                            nn.Conv2d(ngf, ngf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, ngf),
                            nn.BatchNorm2d(ngf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, ngf),
                            nn.LeakyReLU(0.2, inplace=True))

        main.add_module('final_layer.deconv', 
        	             nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False)) # 5,3,1 for 96x96
        main.add_module('final_layer.tanh', 
        	             nn.Tanh())
            # state size. (nc) x 96 x 96

        self.main = main


    def forward(self, input):
        gpu_ids = None
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            gpu_ids = range(self.ngpu)
        return nn.parallel.data_parallel(self.main, input, gpu_ids), 0

In [18]:
netG = _netG_1(ngpu, nz, nc, ngf, n_extra_g)


In [19]:
print(netG)

_netG_1 (
  (main): Sequential (
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (2): LeakyReLU (0.2, inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (5): LeakyReLU (0.2, inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (8): LeakyReLU (0.2, inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (11): LeakyReLU (0.2, inplace)
    (extra-layers-0.64.conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (extra-layers-0.64.batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  

In [20]:
netD = _netD_1(ngpu, nz, nc, ndf, n_extra_d)

In [21]:
print(netD)

_netD_1 (
  (main): Sequential (
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU (0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU (0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (7): LeakyReLU (0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (10): LeakyReLU (0.2, inplace)
    (final_layers.conv): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (final_layers.sigmoid): Sigmoid ()
  )
)


# Carregando pesos pré-treinados

In [22]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


In [23]:
#Parece ser um inicializador de pesos hardcoded
netG.apply(weights_init)
netD.apply(weights_init)
print(())

()


In [24]:
load=False
if load:
    netD.load_state_dict(tc.load('path_d'))
    netG.load_state_dict(torch.load('path_G'))


### Parametros de treinamento

In [25]:
criterion = nn.BCELoss()
criterion_MSE = nn.MSELoss()


In [26]:

input = torch.FloatTensor(batch_size, 3, imageSize, imageSize)
print(input.size())
noise = torch.FloatTensor(batch_size, nz, 1, 1)
print(noise.size())

torch.Size([64, 3, 64, 64])
torch.Size([64, 100, 1, 1])


In [27]:
#parser.add_argument('--binary', action='store_true', help='z from bernoulli distribution, with prob=0.5')
binary=False
#Ele testa pergunta se vc quer que o seu Z venha da distribuição bernoulli
if binary:
    bernoulli_prob = torch.FloatTensor(batch_size, nz, 1, 1).fill_(0.5)
    fixed_noise = torch.bernoulli(bernoulli_prob)
else:
    fixed_noise = torch.FloatTensor(batch_size, nz, 1, 1).normal_(0, 1)


In [28]:
label = torch.FloatTensor(batch_size)
real_label = 1
fake_label = 0


### Broadcast para CUDA, se quiser

In [29]:
if use_gpu:
    netD.cuda()
    netG.cuda()
    criterion = criterion.cuda()
    criterion_MSE = criterion_MSE.cuda()
    input,label = input.cuda(), label.cuda()
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

### Transformando tudo em variable

In [30]:

input = Variable(input)
label = Variable(label)
noise = Variable(noise)
fixed_noise = Variable(fixed_noise)


### Setando o optimizer

In [31]:
beta1, beta2 = 0.9,0.999
lr = 2.0e-4
optimizerD = optim.Adam(netD.parameters(), lr = lr, betas = (beta1, beta2))
optimizerG = optim.Adam(netG.parameters(), lr = lr, betas = (beta1, beta2))


### Criando o diretório vazio

In [45]:
outputDir = 'outputdir_less'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))

## Treinando !

In [32]:
print(type(input))
print(input.size())
print(label.size())

<class 'torch.autograd.variable.Variable'>
torch.Size([64, 3, 64, 64])
torch.Size([64])


In [33]:
def train_both_networks(num_epochs, dataloader, netD, netG, d_labelSmooth, outputDir, model_option =1,binary = False):
    use_gpu = tc.cuda.is_available()

    for epoch in range(num_epochs):
        for i, data in enumerate(dataloader, 0):
            start_iter = time.time()
            ############################
            # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            ###########################
            # train with real
            netD.zero_grad()
            real_cpu, _ = data
            batch_size = real_cpu.size(0)
            input.data.resize_(real_cpu.size()).copy_(real_cpu)
            label.data.resize_(batch_size).fill_(real_label - d_labelSmooth) # use smooth label for discriminator

            output = netD(input)
            errD_real = criterion(output, label)
            errD_real.backward()
            D_x = output.data.mean()
            # train with fake
            noise.data.resize_(batch_size, nz, 1, 1)
            if binary:
                bernoulli_prob.resize_(noise.data.size())
                noise.data.copy_(2*(torch.bernoulli(bernoulli_prob)-0.5))
            else:
                noise.data.normal_(0, 1)
            fake,z_prediction = netG(noise)
            label.data.fill_(fake_label)
            output = netD(fake.detach()) # add ".detach()" to avoid backprop through G
            errD_fake = criterion(output, label)
            errD_fake.backward() # gradients for fake/real will be accumulated
            D_G_z1 = output.data.mean()
            errD = errD_real + errD_fake
            optimizerD.step() # .step() can be called once the gradients are computed

            ############################
            # (2) Update G network: maximize log(D(G(z)))
            ###########################
            netG.zero_grad()
            label.data.fill_(real_label) # fake labels are real for generator cost
            output = netD(fake)
            errG = criterion(output, label)
            errG.backward(retain_variables=True) # True if backward through the graph for the second time
            if model_option == 2: # with z predictor
                errG_z = criterion_MSE(z_prediction, noise)
                errG_z.backward()
            D_G_z2 = output.data.mean()
            optimizerG.step()

            end_iter = time.time()
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f Elapsed %.2f s'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.data[0], errG.data[0], D_x, D_G_z1, D_G_z2, end_iter-start_iter))
            if i % 100 == 0:
                # the first 64 samples from the mini-batch are saved.
                vutils.save_image(real_cpu[0:64,:,:,:],
                        '%s/real_samples.png' % outputDir, nrow=8)
                fake,_ = netG(fixed_noise)
                vutils.save_image(fake.data[0:64,:,:,:],
                        '%s/fake_samples_epoch_%03d.png' % (outputDir, epoch), nrow=8)
        if epoch % 1 == 0:
            # do checkpointing
            torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (outputDir, epoch))
            torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (outputDir, epoch))

In [ ]:
num_epochs = 50
d_labelSmooth = 0.2

train_both_networks(num_epochs, dataloader, netD,netG,d_labelSmooth, outputDir)

/home/gabriel/anaconda3/envs/py35/lib/python3.5/site-packages/torch/nn/functional.py:767: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/home/gabriel/anaconda3/envs/py35/lib/python3.5/site-packages/torch/autograd/__init__.py:92: UserWarning: retain_variables option is deprecated and will be removed in 0.3. Use retain_graph instead.
  warnings.warn("retain_variables option is deprecated and will be removed in 0.3. "


[0/50][0/472] Loss_D: 0.5621 Loss_G: 5.1006 D(x): 0.6846 D(G(z)): 0.0074 / 0.0085 Elapsed 0.08 s
[0/50][1/472] Loss_D: 0.5265 Loss_G: 5.0220 D(x): 0.8248 D(G(z)): 0.0096 / 0.0094 Elapsed 0.06 s
[0/50][2/472] Loss_D: 0.5631 Loss_G: 4.9666 D(x): 0.8771 D(G(z)): 0.0164 / 0.0104 Elapsed 0.06 s
[0/50][3/472] Loss_D: 0.5530 Loss_G: 5.1331 D(x): 0.8617 D(G(z)): 0.0182 / 0.0088 Elapsed 0.06 s
[0/50][4/472] Loss_D: 0.5412 Loss_G: 5.6734 D(x): 0.7434 D(G(z)): 0.0071 / 0.0046 Elapsed 0.06 s
[0/50][5/472] Loss_D: 0.5365 Loss_G: 5.6497 D(x): 0.7574 D(G(z)): 0.0062 / 0.0050 Elapsed 0.06 s
[0/50][6/472] Loss_D: 0.5836 Loss_G: 4.8470 D(x): 0.6550 D(G(z)): 0.0075 / 0.0109 Elapsed 0.06 s
[0/50][7/472] Loss_D: 0.5613 Loss_G: 4.2667 D(x): 0.6841 D(G(z)): 0.0090 / 0.0190 Elapsed 0.06 s
[0/50][8/472] Loss_D: 0.5963 Loss_G: 3.6835 D(x): 0.8914 D(G(z)): 0.0343 / 0.0346 Elapsed 0.06 s
[0/50][9/472] Loss_D: 0.6759 Loss_G: 4.0941 D(x): 0.9393 D(G(z)): 0.0442 / 0.0204 Elapsed 0.06 s
[0/50][10/472] Loss_D: 0.5588 

[0/50][86/472] Loss_D: 0.5467 Loss_G: 4.6964 D(x): 0.7120 D(G(z)): 0.0123 / 0.0116 Elapsed 0.06 s
[0/50][87/472] Loss_D: 0.5331 Loss_G: 5.1503 D(x): 0.8427 D(G(z)): 0.0111 / 0.0081 Elapsed 0.06 s
[0/50][88/472] Loss_D: 0.5299 Loss_G: 5.1957 D(x): 0.7492 D(G(z)): 0.0078 / 0.0068 Elapsed 0.05 s
[0/50][89/472] Loss_D: 0.5571 Loss_G: 4.7731 D(x): 0.6943 D(G(z)): 0.0093 / 0.0113 Elapsed 0.06 s
[0/50][90/472] Loss_D: 0.5355 Loss_G: 4.1892 D(x): 0.7900 D(G(z)): 0.0179 / 0.0193 Elapsed 0.06 s
[0/50][91/472] Loss_D: 0.5968 Loss_G: 4.6059 D(x): 0.9104 D(G(z)): 0.0191 / 0.0128 Elapsed 0.06 s
[0/50][92/472] Loss_D: 0.5549 Loss_G: 5.2033 D(x): 0.8661 D(G(z)): 0.0162 / 0.0077 Elapsed 0.06 s
[0/50][93/472] Loss_D: 0.5285 Loss_G: 5.5367 D(x): 0.7971 D(G(z)): 0.0108 / 0.0052 Elapsed 0.06 s
[0/50][94/472] Loss_D: 0.5692 Loss_G: 5.3400 D(x): 0.6753 D(G(z)): 0.0072 / 0.0061 Elapsed 0.06 s
[0/50][95/472] Loss_D: 0.5851 Loss_G: 4.9175 D(x): 0.6459 D(G(z)): 0.0065 / 0.0098 Elapsed 0.05 s
[0/50][96/472] Loss_

[0/50][169/472] Loss_D: 0.5419 Loss_G: 5.0008 D(x): 0.7543 D(G(z)): 0.0128 / 0.0098 Elapsed 0.06 s
[0/50][170/472] Loss_D: 0.6721 Loss_G: 4.1376 D(x): 0.5553 D(G(z)): 0.0087 / 0.0223 Elapsed 0.06 s
[0/50][171/472] Loss_D: 0.5954 Loss_G: 3.6572 D(x): 0.9005 D(G(z)): 0.0261 / 0.0329 Elapsed 0.06 s
[0/50][172/472] Loss_D: 0.6433 Loss_G: 3.5705 D(x): 0.9114 D(G(z)): 0.0613 / 0.0357 Elapsed 0.06 s
[0/50][173/472] Loss_D: 0.5593 Loss_G: 4.9787 D(x): 0.8715 D(G(z)): 0.0237 / 0.0096 Elapsed 0.06 s
[0/50][174/472] Loss_D: 0.5585 Loss_G: 4.5507 D(x): 0.7600 D(G(z)): 0.0330 / 0.0146 Elapsed 0.06 s
[0/50][175/472] Loss_D: 0.5362 Loss_G: 5.7121 D(x): 0.8064 D(G(z)): 0.0101 / 0.0043 Elapsed 0.06 s
[0/50][176/472] Loss_D: 0.6554 Loss_G: 5.4387 D(x): 0.5686 D(G(z)): 0.0049 / 0.0060 Elapsed 0.06 s
[0/50][177/472] Loss_D: 0.5696 Loss_G: 4.4538 D(x): 0.6747 D(G(z)): 0.0081 / 0.0144 Elapsed 0.06 s
[0/50][178/472] Loss_D: 0.5575 Loss_G: 3.7489 D(x): 0.8576 D(G(z)): 0.0284 / 0.0317 Elapsed 0.06 s
[0/50][179

[0/50][253/472] Loss_D: 0.5500 Loss_G: 5.7791 D(x): 0.7117 D(G(z)): 0.0075 / 0.0047 Elapsed 0.06 s
[0/50][254/472] Loss_D: 0.5581 Loss_G: 5.9539 D(x): 0.6995 D(G(z)): 0.0041 / 0.0036 Elapsed 0.06 s
[0/50][255/472] Loss_D: 0.5406 Loss_G: 5.2614 D(x): 0.7650 D(G(z)): 0.0068 / 0.0075 Elapsed 0.06 s
[0/50][256/472] Loss_D: 0.5301 Loss_G: 5.3511 D(x): 0.8427 D(G(z)): 0.0084 / 0.0071 Elapsed 0.06 s
[0/50][257/472] Loss_D: 0.6051 Loss_G: 3.9693 D(x): 0.6404 D(G(z)): 0.0174 / 0.0284 Elapsed 0.06 s
[0/50][258/472] Loss_D: 0.5806 Loss_G: 3.8234 D(x): 0.8780 D(G(z)): 0.0351 / 0.0304 Elapsed 0.06 s
[0/50][259/472] Loss_D: 0.5975 Loss_G: 4.3315 D(x): 0.8806 D(G(z)): 0.0427 / 0.0184 Elapsed 0.06 s
[0/50][260/472] Loss_D: 0.5431 Loss_G: 5.3071 D(x): 0.7861 D(G(z)): 0.0149 / 0.0070 Elapsed 0.06 s
[0/50][261/472] Loss_D: 0.5604 Loss_G: 5.2605 D(x): 0.7002 D(G(z)): 0.0117 / 0.0082 Elapsed 0.06 s
[0/50][262/472] Loss_D: 0.5679 Loss_G: 4.8227 D(x): 0.6932 D(G(z)): 0.0105 / 0.0116 Elapsed 0.06 s
[0/50][263

...

[31/50][221/472] Loss_D: 0.5324 Loss_G: 4.3368 D(x): 0.7699 D(G(z)): 0.0110 / 0.0188 Elapsed 0.06 s
[31/50][222/472] Loss_D: 0.5870 Loss_G: 3.9809 D(x): 0.8955 D(G(z)): 0.0246 / 0.0253 Elapsed 0.06 s
[31/50][223/472] Loss_D: 0.6174 Loss_G: 4.2391 D(x): 0.9170 D(G(z)): 0.0348 / 0.0204 Elapsed 0.06 s
[31/50][224/472] Loss_D: 0.5412 Loss_G: 5.0240 D(x): 0.7917 D(G(z)): 0.0149 / 0.0089 Elapsed 0.06 s
[31/50][225/472] Loss_D: 0.5310 Loss_G: 5.1574 D(x): 0.8276 D(G(z)): 0.0138 / 0.0075 Elapsed 0.06 s
[31/50][226/472] Loss_D: 0.5273 Loss_G: 5.6942 D(x): 0.7664 D(G(z)): 0.0075 / 0.0047 Elapsed 0.06 s
[31/50][227/472] Loss_D: 0.5967 Loss_G: 5.0135 D(x): 0.6292 D(G(z)): 0.0091 / 0.0101 Elapsed 0.06 s
[31/50][228/472] Loss_D: 0.5276 Loss_G: 5.1033 D(x): 0.8312 D(G(z)): 0.0083 / 0.0081 Elapsed 0.06 s
[31/50][229/472] Loss_D: 0.5302 Loss_G: 4.9664 D(x): 0.7646 D(G(z)): 0.0096 / 0.0097 Elapsed 0.06 s
[31/50][230/472] Loss_D: 0.5583 Loss_G: 4.9086 D(x): 0.8857 D(G(z)): 0.0143 / 0.0103 Elapsed 0.06 s


[31/50][305/472] Loss_D: 0.6096 Loss_G: 5.0522 D(x): 0.6069 D(G(z)): 0.0043 / 0.0090 Elapsed 0.06 s
[31/50][306/472] Loss_D: 0.5349 Loss_G: 3.9281 D(x): 0.8009 D(G(z)): 0.0155 / 0.0251 Elapsed 0.06 s
[31/50][307/472] Loss_D: 0.6051 Loss_G: 4.1251 D(x): 0.9130 D(G(z)): 0.0250 / 0.0229 Elapsed 0.06 s
[31/50][308/472] Loss_D: 0.5500 Loss_G: 4.4973 D(x): 0.8523 D(G(z)): 0.0234 / 0.0157 Elapsed 0.06 s
[31/50][309/472] Loss_D: 0.6223 Loss_G: 5.3386 D(x): 0.9275 D(G(z)): 0.0150 / 0.0066 Elapsed 0.06 s
[31/50][310/472] Loss_D: 0.5343 Loss_G: 5.6531 D(x): 0.8428 D(G(z)): 0.0107 / 0.0043 Elapsed 0.06 s
[31/50][311/472] Loss_D: 0.5764 Loss_G: 6.0451 D(x): 0.6520 D(G(z)): 0.0043 / 0.0033 Elapsed 0.06 s
[31/50][312/472] Loss_D: 0.5948 Loss_G: 5.2779 D(x): 0.6285 D(G(z)): 0.0042 / 0.0064 Elapsed 0.06 s
[31/50][313/472] Loss_D: 0.5448 Loss_G: 4.6976 D(x): 0.7182 D(G(z)): 0.0067 / 0.0122 Elapsed 0.06 s
[31/50][314/472] Loss_D: 0.5743 Loss_G: 4.2874 D(x): 0.8973 D(G(z)): 0.0158 / 0.0179 Elapsed 0.06 s


[31/50][389/472] Loss_D: 0.6060 Loss_G: 6.5241 D(x): 0.6219 D(G(z)): 0.0014 / 0.0019 Elapsed 0.06 s
[31/50][390/472] Loss_D: 0.5775 Loss_G: 4.6746 D(x): 0.6568 D(G(z)): 0.0066 / 0.0135 Elapsed 0.06 s
[31/50][391/472] Loss_D: 0.5424 Loss_G: 4.2570 D(x): 0.8395 D(G(z)): 0.0127 / 0.0190 Elapsed 0.06 s
[31/50][392/472] Loss_D: 0.5728 Loss_G: 3.5144 D(x): 0.8663 D(G(z)): 0.0371 / 0.0366 Elapsed 0.06 s
[31/50][393/472] Loss_D: 0.5805 Loss_G: 4.2905 D(x): 0.8884 D(G(z)): 0.0295 / 0.0199 Elapsed 0.06 s
[31/50][394/472] Loss_D: 0.5604 Loss_G: 4.6348 D(x): 0.8448 D(G(z)): 0.0260 / 0.0136 Elapsed 0.06 s
[31/50][395/472] Loss_D: 0.5412 Loss_G: 4.9970 D(x): 0.8163 D(G(z)): 0.0194 / 0.0093 Elapsed 0.06 s
[31/50][396/472] Loss_D: 0.5259 Loss_G: 5.7981 D(x): 0.7580 D(G(z)): 0.0076 / 0.0043 Elapsed 0.06 s
[31/50][397/472] Loss_D: 0.7186 Loss_G: 4.9179 D(x): 0.5085 D(G(z)): 0.0057 / 0.0095 Elapsed 0.06 s
[31/50][398/472] Loss_D: 0.6057 Loss_G: 4.0819 D(x): 0.6191 D(G(z)): 0.0085 / 0.0213 Elapsed 0.06 s


[32/50][0/472] Loss_D: 0.5493 Loss_G: 4.7734 D(x): 0.8564 D(G(z)): 0.0192 / 0.0111 Elapsed 0.08 s
[32/50][1/472] Loss_D: 0.5497 Loss_G: 4.9112 D(x): 0.8540 D(G(z)): 0.0224 / 0.0093 Elapsed 0.06 s
[32/50][2/472] Loss_D: 0.5628 Loss_G: 5.1967 D(x): 0.6829 D(G(z)): 0.0106 / 0.0077 Elapsed 0.06 s
[32/50][3/472] Loss_D: 0.5839 Loss_G: 5.1449 D(x): 0.6563 D(G(z)): 0.0051 / 0.0075 Elapsed 0.06 s
[32/50][4/472] Loss_D: 0.5427 Loss_G: 4.2243 D(x): 0.7471 D(G(z)): 0.0124 / 0.0187 Elapsed 0.06 s
[32/50][5/472] Loss_D: 0.5527 Loss_G: 3.6315 D(x): 0.8150 D(G(z)): 0.0351 / 0.0364 Elapsed 0.06 s
[32/50][6/472] Loss_D: 0.5742 Loss_G: 4.0699 D(x): 0.8619 D(G(z)): 0.0369 / 0.0245 Elapsed 0.06 s
[32/50][7/472] Loss_D: 0.5409 Loss_G: 4.6324 D(x): 0.8091 D(G(z)): 0.0240 / 0.0144 Elapsed 0.06 s
[32/50][8/472] Loss_D: 0.5330 Loss_G: 5.2699 D(x): 0.7902 D(G(z)): 0.0105 / 0.0063 Elapsed 0.06 s
[32/50][9/472] Loss_D: 0.5329 Loss_G: 5.1111 D(x): 0.8261 D(G(z)): 0.0135 / 0.0078 Elapsed 0.06 s
[32/50][10/472] Loss

[32/50][85/472] Loss_D: 0.5378 Loss_G: 4.8794 D(x): 0.7473 D(G(z)): 0.0104 / 0.0096 Elapsed 0.06 s
[32/50][86/472] Loss_D: 0.5313 Loss_G: 4.8005 D(x): 0.7704 D(G(z)): 0.0108 / 0.0106 Elapsed 0.06 s
[32/50][87/472] Loss_D: 0.5346 Loss_G: 4.6143 D(x): 0.8223 D(G(z)): 0.0146 / 0.0123 Elapsed 0.06 s
[32/50][88/472] Loss_D: 0.5352 Loss_G: 4.6800 D(x): 0.8075 D(G(z)): 0.0157 / 0.0118 Elapsed 0.06 s
[32/50][89/472] Loss_D: 0.5286 Loss_G: 5.6551 D(x): 0.7730 D(G(z)): 0.0054 / 0.0045 Elapsed 0.06 s
[32/50][90/472] Loss_D: 0.5255 Loss_G: 5.7435 D(x): 0.8112 D(G(z)): 0.0050 / 0.0041 Elapsed 0.06 s
[32/50][91/472] Loss_D: 0.5426 Loss_G: 4.8486 D(x): 0.7195 D(G(z)): 0.0092 / 0.0095 Elapsed 0.06 s
[32/50][92/472] Loss_D: 0.5320 Loss_G: 5.0534 D(x): 0.8220 D(G(z)): 0.0102 / 0.0087 Elapsed 0.06 s
[32/50][93/472] Loss_D: 0.5462 Loss_G: 4.4492 D(x): 0.7570 D(G(z)): 0.0160 / 0.0150 Elapsed 0.06 s
[32/50][94/472] Loss_D: 0.5391 Loss_G: 4.7300 D(x): 0.7799 D(G(z)): 0.0119 / 0.0112 Elapsed 0.06 s
[32/50][95